In [1]:
import pandas as pd
import numpy as np

### 피처셀렉
1. t.test로 p-value가 0.05보다 작은 값 => 유의미한 피처로 선택
2. lasso

In [8]:
df = pd.read_csv("train_data.csv",index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1566 entries, 2.4 to -2.98
Data columns (total 66 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   총자본순이익률_비율          1566 non-null   float64
 1   총자본순이익률_상위10개_차이    1566 non-null   float64
 2   총자본순이익률_상위10개_변동성   1566 non-null   float64
 3   총자본순이익률_산업평균_차이     1566 non-null   float64
 4   총자본순이익률_산업평균_변동성    1566 non-null   float64
 5   매출액증가율_차이           1566 non-null   float64
 6   매출액증가율_비율           1566 non-null   float64
 7   매출액증가율_상위10개_차이     1566 non-null   float64
 8   매출액증가율_상위10개_변동성    1566 non-null   float64
 9   매출액증가율_산업평균_차이      1566 non-null   float64
 10  매출액증가율_산업평균_변동성     1566 non-null   float64
 11  자기자본순이익률_차이         1566 non-null   float64
 12  자기자본순이익률_비율         1566 non-null   float64
 13  자기자본순이익률_상위10개_차이   1566 non-null   float64
 14  자기자본순이익률_상위10개_변동성  1566 non-null   float64
 15  자기자본순이익률_산업평균_차이    1566 non-null   float64
 16  자기자본순이익률

In [9]:
# H가 1인 경우 이분산성 / H가 0인 경우 등분산
from scipy.stats import bartlett
def bartlett_test(col, p_value = 0.05, H = 1):
    list= []
    for i in col:
        T, p_val =bartlett(df[df['label']==1][i], df[df['label']==0][i]) 
        list.append([i, p_val])

    list = pd.DataFrame(list, columns = ['변수', 'p_value'])
    if H == 1:
        a = list[(list['p_value'] < p_value)][['변수', 'p_value']].sort_values('p_value')
        return a
    else:
        a = list[(list['p_value'] >= p_value)][['변수', 'p_value']].sort_values('p_value')
        return a

In [10]:
# 이분산성 변수
x_hetero = bartlett_test(df.columns, H = 1)
# 등분산성 변수
x_homo = bartlett_test(df.columns, H = 0)

c:\Users\dlsgm\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:2901: RuntimeWarning: divide by zero encountered in log
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\Users\dlsgm\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:2901: RuntimeWarning: invalid value encountered in scalar subtract
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\Users\dlsgm\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:2901: RuntimeWarning: divide by zero encountered in log
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\Users\dlsgm\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_morestats.py:2901: RuntimeWarning: invalid value encountered in scalar subtract
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)


In [12]:
# t-test

import scipy.stats as stats
def t_test(col, col_h0, col_h1, p_value = 0.05):
    list= []
    for i in col:
        if (col_h0['변수']==i).any():
            t_stat, p_val = stats.ttest_ind(df[df['label']==1][i], df[df['label']==0][i], equal_var=True) # 등분산성 : wald t-test
            list.append([i, p_val])
        elif (col_h1['변수']==i).any():
            t_stat, p_val = stats.ttest_ind(df[df['label']==1][i], df[df['label']==0][i], equal_var=False) # 이분산성 : welch’s t-test
            list.append([i, p_val])

    list = pd.DataFrame(list, columns = ['변수', 'p_value'])
    a = list[(list['p_value'] < p_value)][['변수', 'p_value']].sort_values('p_value')
    return a

In [13]:
# 2) t_test 결과 p_value < 0.05보다 작은 유의한 변수 가져오기
x_ttest = t_test(df.columns, x_homo, x_hetero, p_value=0.1)
print("유의한 피쳐 수 :", len(x_ttest))
x_ttest.sort_values(by="변수", ascending=True)

유의한 피쳐 수 : 42


,변수,p_value
18,매출액순이익률_비율,2.338277e-05
22,매출액순이익률_산업평균_변동성,6.257483e-03
21,매출액순이익률_산업평균_차이,5.766874e-04
20,매출액순이익률_상위10개_변동성,3.094198e-03
19,매출액순이익률_상위10개_차이,3.534007e-03
6,매출액증가율_비율,8.769194e-04
10,매출액증가율_산업평균_변동성,8.994816e-04
9,매출액증가율_산업평균_차이,4.572676e-02
5,매출액증가율_차이,6.852767e-02
60,순이익증가율_비율,4.915805e-08


In [19]:
ttest = list(x_ttest['변수'])
ttest

['유동비율_비율',
 '자기자본회전률_비율',
 '자기자본순이익률_비율',
 '순이익증가율_비율',
 '총자본투자효율_산업평균_차이',
 '총자본회전률_상위10개_차이',
 '총자본회전률_비율',
 '총자본회전률_상위10개_변동성',
 '총자본투자효율_상위10개_차이',
 '총자본투자효율_상위10개_변동성',
 '순이익증가율_산업평균_변동성',
 '총자본순이익률_비율',
 '총자본회전률_산업평균_차이',
 '자기자본구성비율_비율',
 '총자본순이익률_산업평균_차이',
 '총자본순이익률_상위10개_변동성',
 '총자본순이익률_상위10개_차이',
 '총자본순이익률_산업평균_변동성',
 '자기자본구성비율_산업평균_변동성',
 '매출액순이익률_비율',
 '총자본투자효율_산업평균_변동성',
 '자기자본회전률_상위10개_차이',
 '자기자본순이익률_상위10개_차이',
 '자기자본회전률_상위10개_변동성',
 '자기자본순이익률_상위10개_변동성',
 '매출액순이익률_산업평균_차이',
 '자기자본구성비율_상위10개_차이',
 '자기자본구성비율_상위10개_변동성',
 '자기자본순이익률_산업평균_차이',
 '자기자본구성비율_산업평균_차이',
 '매출액증가율_비율',
 '매출액증가율_산업평균_변동성',
 '매출액순이익률_상위10개_변동성',
 '매출액순이익률_상위10개_차이',
 '매출액순이익률_산업평균_변동성',
 '자기자본회전률_산업평균_차이',
 '순이익증가율_산업평균_차이',
 '자기자본순이익률_차이',
 '매출액증가율_산업평균_차이',
 '매출액증가율_차이',
 '총자본투자효율_비율',
 '유동비율_상위10개_변동성']

In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.simplefilter('ignore')

# 예시 데이터셋 (이 부분은 실제 데이터에 맞게 수정해야 함)
X = df.drop('label', axis=1) # 전체 특성 데이터
y = df['label'] # 레이블 데이터

# 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 로지스틱 회귀 모델 설정
lasso_model = LogisticRegression()
param_grid = {'penalty': ['l1'], 
              'C': [0.001, 0.01, 0.1, 1, 2, 5, 10],
              'solver': ['liblinear']}

# 그리드 서치
grid_search = GridSearchCV(lasso_model, param_grid=param_grid, return_train_score=True, cv=5)
grid_search.fit(X_train, y_train)

# 결과 DataFrame 생성 및 정렬
df = pd.DataFrame(grid_search.cv_results_)
df = df.sort_values(by=['rank_test_score'], ascending=True)
df[['params', 'mean_train_score', 'mean_test_score', 'rank_test_score']]

# 최적의 파라미터와 정확도 출력
print('GridSearchCV 최적 파라미터:', grid_search.best_params_)
print('GridSearchCV 최고 정확도:{0:.4f}'.format(grid_search.best_score_))


GridSearchCV 최적 파라미터: {'C': 0.001, 'penalty': 'l1', 'solver': 'liblinear'}
GridSearchCV 최고 정확도:0.9717


In [17]:
lasso_best = LogisticRegression(C=0.001, penalty='l1', solver='liblinear').fit(X_train, y_train)

df_lasso = pd.DataFrame()
df_lasso['feature'] = X_train.columns
df_lasso['coef'] = lasso_best.coef_[0]
df_lasso.drop(df_lasso[df_lasso['coef']==0].index, inplace=True)
df_lasso

,feature,coef
15,자기자본순이익률_산업평균_차이,-0.000736
19,매출액순이익률_상위10개_차이,-0.003831
20,매출액순이익률_상위10개_변동성,0.000248
27,유동비율_산업평균_차이,-0.002025
28,유동비율_산업평균_변동성,0.001186
53,설비투자효율_차이,0.000252
55,설비투자효율_상위10개_차이,-0.000452
56,설비투자효율_상위10개_변동성,-0.000593
57,설비투자효율_산업평균_차이,-0.000304
58,설비투자효율_산업평균_변동성,0.000319


In [18]:
# 라쏘에서 선택된 피처
lasso = df_lasso['feature'].values.tolist()
print('Lasso에서 선택된 피처 수 {0:1.0f}'.format(len(df_lasso)), '개')
lasso

Lasso에서 선택된 피처 수 13 개


['자기자본순이익률_산업평균_차이',
 '매출액순이익률_상위10개_차이',
 '매출액순이익률_상위10개_변동성',
 '유동비율_산업평균_차이',
 '유동비율_산업평균_변동성',
 '설비투자효율_차이',
 '설비투자효율_상위10개_차이',
 '설비투자효율_상위10개_변동성',
 '설비투자효율_산업평균_차이',
 '설비투자효율_산업평균_변동성',
 '순이익증가율_비율',
 '순이익증가율_상위10개_변동성',
 '순이익증가율_산업평균_변동성']

In [23]:
list_ttest= list(ttest)
list_lasso = list(lasso)
list_col_all = X_train.columns

In [24]:
def func_ttest(x):
    if x in list_ttest:
        return 1
    else:
        return 0
    
def func_lasso(x):
    if x in list_lasso:
        return 1
    else:
        return 0

In [25]:
# 2번 선택된 Feature

feature_counts = pd.DataFrame()
feature_counts['Feature'] = list_col_all
feature_counts['ttest'] = list_col_all.map(func_ttest)
feature_counts['lasso'] = list_col_all.map(func_lasso)

feature_counts["total"] = feature_counts["ttest"]+feature_counts["lasso"]
feature_final = feature_counts[feature_counts["total"]>=2]
list_feature_final = list(feature_final["Feature"])
print("선택된 피쳐수 :", len(list_feature_final))
feature_final

선택된 피쳐수 : 5


,Feature,ttest,lasso,total
15,자기자본순이익률_산업평균_차이,1,1,2
19,매출액순이익률_상위10개_차이,1,1,2
20,매출액순이익률_상위10개_변동성,1,1,2
60,순이익증가율_비율,1,1,2
64,순이익증가율_산업평균_변동성,1,1,2
